# Classifying GOES Imagery With TensorFlow

We just built a simple logistic regression with a single hidden layer using `numpy` arrays, but there are libraries available that can make building more complex models more efficient. One such library is TensorFlow. A lot of the courses I took on machine learning used tensorflow, but since a lot of the code was already provided, I didn't learn much. Here, I want to build a slightly deeper model using TensorFlow. I still expect that we need a convolutional neural network to get anything useful for our image classification, so this is more of a tool to familiarize myself with TensorFlow than a valuable model for image classification.

## Import modules

First, let's import the modules we need. Some of the functions created when I built the logistic regression model have been placed in `utils`.

In [2]:
import tensorflow as tf
import utils
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import Model

## Load Data

As before, we now load the data as numpy arrays. Note that some of the data formatting has changed slightly, to match the TensorFlow documentation.

In [3]:
# Build the list to load training and testing datasets
xTrainList,yTrainList,xTestList,yTestList = utils.build_dataset(1000,0.7)
print("Training data size:",len(xTrainList))
print("Testing data size:",len(xTestList))

# Load the data arrays
xTrain, yTrain = utils.load_data(xTrainList,yTrainList)
muTrain, sigmaTrain, xTrain = utils.normalize_samples(xTrain)
xTest, yTest = utils.load_data(xTestList,yTestList)
muTest, sigmaTest, xTest = utils.normalize_samples(xTest)
print("Shape of xTrain: "+str(xTrain.shape))
print("Shape of yTrain: "+str(yTrain.shape))
print("Shape of xTest: "+str(xTest.shape))
print("Shape of yTest: "+str(yTest.shape))

Training data size: 700
Testing data size: 300
Shape of xTrain: (700, 498, 498, 1)
Shape of yTrain: (700,)
Shape of xTest: (300, 498, 498, 1)
Shape of yTest: (300,)


Now we use `tf.data` to load, shuffle, and batch the data.

In [4]:
dsTrain = tf.data.Dataset.from_tensor_slices((xTrain,yTrain)).shuffle(10000).batch(10)
dsTest = tf.data.Dataset.from_tensor_slices((xTest, yTest)).batch(10)

In [5]:
class MyModel(Model):
    
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.maxpool1 = MaxPooling2D((2, 2))
        self.conv2 = Conv2D(64, 3, activation='relu')
        self.maxpool2 = MaxPooling2D((2, 2))
        self.conv3 = Conv2D(34, 3, activation='relu')
        self.maxpool3 = MaxPooling2D((2, 2))
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(2)

    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.maxpool3(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model
model = MyModel()

# Define the loss function and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

# Define the metrics for loss and accuracy
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [6]:
@tf.function
def train_step(images, labels):
    
    with tf.GradientTape() as tape:    
        # training=True is only needed if there are layers with different behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        train_loss(loss)
        train_accuracy(labels, predictions)
    
@tf.function
def test_step(images, labels):
    
    # training=False is only needed if there are layers with different behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [7]:
EPOCHS = 10

for epoch in range(EPOCHS):
    
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in dsTrain:
        train_step(images, labels)

    for test_images, test_labels in dsTest:
        test_step(test_images, test_labels)

    print(f'Epoch {epoch + 1}, '
          f'Loss: {train_loss.result()}, '
          f'Accuracy: {train_accuracy.result() * 100}, '
          f'Test Loss: {test_loss.result()}, '
          f'Test Accuracy: {test_accuracy.result() * 100}')

Epoch 1, Loss: 0.47923165559768677, Accuracy: 82.71428680419922, Test Loss: 0.41239938139915466, Test Accuracy: 87.66666412353516
Epoch 2, Loss: 0.26429906487464905, Accuracy: 90.85713958740234, Test Loss: 0.2567271292209625, Test Accuracy: 91.0
Epoch 3, Loss: 0.15227936208248138, Accuracy: 95.28571319580078, Test Loss: 0.272224098443985, Test Accuracy: 89.33332824707031
Epoch 4, Loss: 0.08242031186819077, Accuracy: 96.71428680419922, Test Loss: 0.21976238489151, Test Accuracy: 94.0
Epoch 5, Loss: 0.015454903244972229, Accuracy: 99.42857360839844, Test Loss: 0.27267712354660034, Test Accuracy: 92.66666412353516
Epoch 6, Loss: 0.005630664527416229, Accuracy: 100.0, Test Loss: 0.3071818947792053, Test Accuracy: 93.0
Epoch 7, Loss: 0.0006477537681348622, Accuracy: 100.0, Test Loss: 0.2896701991558075, Test Accuracy: 94.66666412353516
Epoch 8, Loss: 0.00012878127745352685, Accuracy: 100.0, Test Loss: 0.29674622416496277, Test Accuracy: 94.66666412353516
Epoch 9, Loss: 8.131034701364115e-05